## Análisis de sentimientos expresados en noticias WEB

In [1]:
### ######################################
### Sentiment Analysis sobre noticias web
### Jose Maria de Cuenca
### ######################################

# Carga librerias y funciones
import re # Librería para manejo de expresiones regulares en el lenguaje natural
import string # Librería para manejo de cadenas
import nltk # Librería para procesamiento del lenguaje natural

from nltk.corpus import stopwords # Función adicional NLTK para identificar palabras irrelevantes
from textblob import TextBlob # Funcion para análisis de sentimientos
from urllib.request import urlopen as uReq # Permite hacer requerimientos a webs
from bs4 import BeautifulSoup as soup # Funcion para extraer datos desde textos en HTML y XML

### Carga de la prensa a analizar

In [2]:
# Defino enlace a pagina de noticias
# Idealmente la hemeroteca de un periodo local, que hable de nosotros, con noticias ordenadas por fecha
# Para este ejemplo, el decano de la prensa en la ciudad de Valladolid, y el servicio de aguas en ella, Aquavall.
my_url = 'https://www.elnortedecastilla.es/hemeroteca/aquavall.html?order=-fecha'

# Utilizamos la libreria urlopen (as uReq) de urllib.request para abrir sesion y leer las noticias
uClient = uReq(my_url)
page_html = uClient.read()

# Cierro la conexion a la pagina
uClient.close()

# Cargo el contenido de la pagina en un objeto soup, realizando su análisis sintáctico para HTML
page_soup = soup(page_html, 'html.parser')

In [3]:
# El objeto soup permite localizar cada seccion de la página HTML, por ejemplo títulos, etiquetas, etc.
print(page_soup)

<!DOCTYPE html>
<html lang="es-ES"><head><meta charset="utf-8"/><script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={xpid:"VQcDVFBRCRACXVRUBQQGUlQ=",licenseKey:"c72ce647f9",applicationID:"295625140"};window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var i=e[n]={exports:{}};t[n][0].call(i.exports,function(e){var i=t[n][1][e];return r(i||e)},i,i.exports)}return e[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var i=0;i<n.length;i++)r(n[i]);return r}({1:[function(t,e,n){function r(t){try{c.console&&console.log(t)}catch(e){}}var i,o=t("ee"),a=t(23),c={};try{i=localStorage.getItem("__nr_flags").split(","),console&&"function"==typeof console.log&&(c.console=!0,i.indexOf("dev")!==-1&&(c.dev=!0),i.indexOf("nr_dev")!==-1&&(c.nrDev=!0))}catch(s){}c.nrDev&&o.on("internal-error",function(t){r(t.stack)}),c.dev&&o.on("fn-err",function(t,e,n){r(n.stack)}),c.dev&&(r("NR AGENT IN DEVELOPMENT MODE"),r("flags: "+a(c,function(t,e){retu

### Preparación del contenido HTML para su análisis

In [4]:
# El texto HTML almacenado contiene todo el texto de la página, con muchas secciones irrelevantes para el análisis
# El análisis de sentimientos se centrará solamente en los titulares recuperados de la hemeroteca

# En este ejemplo, se usa la cabecera de tipo 'h2' para maquetar la lista de titulares de noticias
# por tanto seleccionaremos solo las partes de código HTML que son cabeceras 'h2'
# Esto puede variar si se analiza una pagina diferente, con una maquetación distinta
news_general = page_soup.findAll('h2')

print(news_general)


[<h2 class="voc-hemeroteca-title-inside"><strong>Resultados</strong> para <span class="topic">'aquavall'</span></h2>, <h2 class="voc-title"><a alt="Mensajes de apoyo a los vallisoletanos" href="https://www.elnortedecastilla.es/valladolid/fiestas/mensajes-apoyo-vallisoletanos-20200908183139-nt.html" title="Mensajes de apoyo a los vallisoletanos">
                Mensajes de apoyo a los vallisoletanos
            </a></h2>, <h2 class="voc-title"><a alt="Aquavall rehabilita con tecnología sin zanja la red de alcantarillado de Valladolid" href="https://www.elnortedecastilla.es/valladolid/aquavall-rehabilita-tecnologia-20200820121334-nt.html" title="Aquavall rehabilita con tecnología sin zanja la red de alcantarillado de Valladolid">
                Aquavall rehabilita con tecnología sin zanja la red de alcantarillado de Valladolid
            </a></h2>, <h2 class="voc-title"><a alt="El reventón de una tubería obliga a cortar la calle Olmo de Valladolid" href="https://www.elnortedecastilla.

In [5]:
# Los titulares de las noticias que muestra una consulta a la hemeroteca continenen enlaces a ellas
# En HTML los hipervínculos se indican con el marcador <a....></a>
# Como el objeto soup mantienen la estructura de la página, podemos extraerlos con ese marcador HTML

# Creamos un objeto de texto para el contenido
news_data = []

for news in news_general:
    news1 = news.find('a')
    news_data.append([news1])

print(news_data)

[[None], [<a alt="Mensajes de apoyo a los vallisoletanos" href="https://www.elnortedecastilla.es/valladolid/fiestas/mensajes-apoyo-vallisoletanos-20200908183139-nt.html" title="Mensajes de apoyo a los vallisoletanos">
                Mensajes de apoyo a los vallisoletanos
            </a>], [<a alt="Aquavall rehabilita con tecnología sin zanja la red de alcantarillado de Valladolid" href="https://www.elnortedecastilla.es/valladolid/aquavall-rehabilita-tecnologia-20200820121334-nt.html" title="Aquavall rehabilita con tecnología sin zanja la red de alcantarillado de Valladolid">
                Aquavall rehabilita con tecnología sin zanja la red de alcantarillado de Valladolid
            </a>], [<a alt="El reventón de una tubería obliga a cortar la calle Olmo de Valladolid" href="https://www.elnortedecastilla.es/valladolid/operarios-trabajan-reparar-20200805105125-nt.html" title="El reventón de una tubería obliga a cortar la calle Olmo de Valladolid">
                El reventón de una 

### Extracción de palabras relevantes desde los titulares de noticias

In [6]:
# Para facilitar el análisis limpiaremos el texto, simplificándolo y eliminando signos de puntuacion
# Además, para reducir las variaciones, ponemos todas las letras en minusculas

# En primer lugar definimos un modelo que obvie los signos de puntuación usados en el lenguaje natural
# Usamos la libreria de expresiones regulares re. Usaremos esta funcion más adelante dentro del bucle
# Añado algunos caracteres que no contempla la librería
regex = re.compile('[%s]' % re.escape((string.punctuation) + u'¡' + u'¿' + u'«'+ u'»'))


# Creamos un objeto lista vacío para poner todas las palabras extraidas
data_new = []

# Recorremos la selección con las cabeceras de noticias para extraer las palabras
# Recorremos de una en una las lineas, seleccionando las que no sean nulas
for new in news_data:
    if not 'None' in new:
        for sentence in new:
            if sentence is None:
                pass
            else:
                # Si la línea no es nula, partimos su contenido en palabras
                for words in sentence:
                    words = words.split()
                    # Una vez extraidas las palabras, las procesamos
                    for word in words:
                        # Ponemos todos los caracteres de cada palabra en minúsculas
                        token = word.lower()
                        # Eliminamos los signos de puntuacion sustituyéndolos (sub) por nada
                        new_token1 = regex.sub(u'', token)
                        # Incorporamos esa palabra al objeto lista que creamos inicialmente
                        data_new.append(new_token1)

In [7]:
# El contenido del objeto lista con las palabras que contienen los titulares de noticias es
print(data_new)

['mensajes', 'de', 'apoyo', 'a', 'los', 'vallisoletanos', 'aquavall', 'rehabilita', 'con', 'tecnología', 'sin', 'zanja', 'la', 'red', 'de', 'alcantarillado', 'de', 'valladolid', 'el', 'reventón', 'de', 'una', 'tubería', 'obliga', 'a', 'cortar', 'la', 'calle', 'olmo', 'de', 'valladolid', 'valladolid', 'analiza', 'todos', 'los', 'domingos', 'sus', 'aguas', 'residuales', 'para', 'buscar', 'trazas', 'de', 'coronavirus', 'los', 'vallisoletanos', 'podrán', 'pagar', 'los', 'impuestos', 'municipales', 'en', 'correos', 'el', 'ayuntamiento', 'diseña', 'un', 'plan', 'para', 'prevenir', 'y', 'erradicar', 'el', 'acoso', 'laboral', 'entre', 'sus', 'empleados', 'un', 'reventón', 'anega', 'dos', 'locales', 'y', 'deja', '15', 'horas', 'sin', 'agua', 'dos', 'calles', 'de', 'la', 'rondilla', 'en', 'valladolid', 'el', 'reventón', 'de', 'una', 'tubería', 'en', 'la', 'rondilla', 'anega', 'varios', 'locales', 'de', 'la', 'calle', 'linares', 'los', 'parques', 'infantiles', 'de', 'valladolid', 'reabrirán', 'el

In [8]:
# Deseamos solo las palabras relevantes, las que determinan el sentido y sentimientos de sus frases

# Los pronombres, artículos o preposiciones no son relevantes para determinar el sentimiento
# Podemos extraer las palabras relevantes del lenguaje natural mediante la librería NLTK

# Descargo las stopwords actualizadas de la libreria NLTK
nltk.download('stopwords') # Descargo su lista de palabras irrelevantes (stopwords)

# Selecciono solo las palabras irrelevantes (stopwords) del idioma español
stop_words = set(stopwords.words('spanish'))

# Añado algunas palabras que en mi caso particular no serán relevantes
stop_words.update(('valladolid', 'vallisoletano', 'aquavall', 'agua', 'aguas'))

# Creo un objeto lista de palabras relevantes
data_no_stopwords = []

# Recorro la lista de palabras extraidas anteriormente de los titulaes para dejar solo las relevantes
for word in data_new:
    if not word in stop_words:
        data_no_stopwords.append(word)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jose\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
# Muestro el resultado de la lista de palabras relevantes
print(data_no_stopwords)

['mensajes', 'apoyo', 'vallisoletanos', 'rehabilita', 'tecnología', 'zanja', 'red', 'alcantarillado', 'reventón', 'tubería', 'obliga', 'cortar', 'calle', 'olmo', 'analiza', 'domingos', 'residuales', 'buscar', 'trazas', 'coronavirus', 'vallisoletanos', 'podrán', 'pagar', 'impuestos', 'municipales', 'correos', 'ayuntamiento', 'diseña', 'plan', 'prevenir', 'erradicar', 'acoso', 'laboral', 'empleados', 'reventón', 'anega', 'dos', 'locales', 'deja', '15', 'horas', 'dos', 'calles', 'rondilla', 'reventón', 'tubería', 'rondilla', 'anega', 'varios', 'locales', 'calle', 'linares', 'parques', 'infantiles', 'reabrirán', 'martes', 'promueve', 'consumo', 'grifo', 'hosteleros']


### Análisis automatico de sentimientos con librería TextBlob (para idioma ingles)

In [10]:
## ANALISIS DE SENTIMIENTOS

# En ingles se usa la libreria TextBlob para analizar la polaridad de cada palabra en ingles
# La polaridad modifica el valor de sentimeniento que dará al final el texto
# Pero en español esta libreria no tiene una clasificacion de palabras, hay que traducirlas

# En primer lugar inicializamos el marcador para la puntuación de sentimientos
sentiment = 0

# Recorro la lista traduciendo de una en una al inglés (trabaja con palabras o frases, no con listas)

for word in data_no_stopwords:
    # Traduzco la palabra del español al inglés y analizo el sentimiento
    # Para evitar que se interrumpa la ejecución en caso de error defino control de intento
    try:
        # El resultado del análisis es un objeto TextBlob
        analysis = TextBlob(word).translate(from_lang='es', to='en')
    except:
        # si no puede traducir la palabra, la analizo sin traducir
        analysis = TextBlob(word)
    
    # Muestro como trabaja el análisis
    print(word, " - ", analysis, " - ", analysis.sentiment.polarity)
    
    # Analizo el 'objeto TextBlob' con la palabra traducida
    if analysis.sentiment.polarity > 0:
        sentiment += 1
    elif analysis.sentiment.polarity == 0:
        sentiment = sentiment
    else:
        sentiment -= 1
        
print("Puntuación final: ", sentiment)

mensajes  -  posts  -  0.0
apoyo  -  support for  -  0.0
vallisoletanos  -  Valladolid  -  0.0
rehabilita  -  rehabilitates  -  0.0
tecnología  -  technology  -  0.0
zanja  -  ditch  -  0.0
red  -  net  -  0.0
alcantarillado  -  sewerage  -  0.0
reventón  -  blowout  -  0.0
tubería  -  pipeline  -  0.0
obliga  -  obliges  -  0.0
cortar  -  cut  -  0.0
calle  -  Street  -  0.0
olmo  -  elm  -  0.0
analiza  -  analyze  -  0.0
domingos  -  Sundays  -  0.0
residuales  -  residual  -  0.0
buscar  -  search for  -  0.0
trazas  -  traces  -  0.0
coronavirus  -  coronavirus  -  0.0
vallisoletanos  -  Valladolid  -  0.0
podrán  -  will be able  -  0.5
pagar  -  pay  -  0.0
impuestos  -  taxes  -  0.0
municipales  -  municipal  -  0.0
correos  -  post  -  0.0
ayuntamiento  -  town hall  -  0.0
diseña  -  designs  -  0.0
plan  -  plan  -  0.0
prevenir  -  to prevent  -  0.0
erradicar  -  eradicate  -  0.0
acoso  -  bullying  -  0.0
laboral  -  labor  -  0.0
empleados  -  employees  -  0.0
reventó

### Análisis de sentimientos con nuestra propia lista de palabras en español

Vemos que la puntuación final es nuetra, porque ha encontrado el mismo número de palabras que una vez traducidas considera negativas, que de positivas.

Para mejorar el análisis en español, y en una aplicación específica, lo mejor es definir nuestras propias listas de palabras.

In [11]:
# Creamos listas de palabras positivas y negativas para valorar las noticias relativas al sector del agua
pal_posit = ['bueno', 'excelente', 'rapido', 'eficaz', 'eficiente', 'restablecer', 'restablece', 'recuperar', 'recupera', 'concluir', 'invertir', 'inversión', 'invierte', 'mejora', 'reforma', 'primera', 'multiplica', 'reurbaniza', 'rehabilita', 'valor', 'valorizar', 'revalorizar', 'sostenible', 'gana', 'beneficios', 'beneficia', 'prevenir', 'diseñar', 'diseña', 'analiza', 'promover', 'promueve', 'tecnología']
pal_neg = ['averia', 'reventón', 'rotura', 'inundacion', 'inundar', 'inunda', 'fuga', 'balsa', 'malo', 'peor', 'corte', 'cortar', 'desabastecido', 'desastre', 'desastroso', 'suspenso', 'suspendidas', 'equivocación', 'equivocarse', 'error', 'errar', 'imprevisto', 'poco', 'poca', 'falta', 'obliga', 'impuestos']

# Creamos una lista ordenada con todos los elementos unicos que aparecen en el texto original
palabras = sorted(list(set(data_no_stopwords)))

print(palabras)

['15', 'acoso', 'alcantarillado', 'analiza', 'anega', 'apoyo', 'ayuntamiento', 'buscar', 'calle', 'calles', 'consumo', 'coronavirus', 'correos', 'cortar', 'deja', 'diseña', 'domingos', 'dos', 'empleados', 'erradicar', 'grifo', 'horas', 'hosteleros', 'impuestos', 'infantiles', 'laboral', 'linares', 'locales', 'martes', 'mensajes', 'municipales', 'obliga', 'olmo', 'pagar', 'parques', 'plan', 'podrán', 'prevenir', 'promueve', 'reabrirán', 'red', 'rehabilita', 'residuales', 'reventón', 'rondilla', 'tecnología', 'trazas', 'tubería', 'vallisoletanos', 'varios', 'zanja']


In [12]:
# Comparamos entre listas
loc_posit = set(palabras).intersection(pal_posit)
print("Palabras positivas localizadas: ", loc_posit)

loc_neg = set(palabras).intersection(pal_neg)
print("Palabras positivas localizadas: ", loc_neg)

Palabras positivas localizadas:  {'tecnología', 'promueve', 'rehabilita', 'prevenir', 'analiza', 'diseña'}
Palabras positivas localizadas:  {'reventón', 'cortar', 'obliga', 'impuestos'}


In [13]:
# Obtenemos la puntuación como diferencia de longitudes entre las listas de palabras localizadas
sentimiento = len(loc_posit)-len(loc_neg)
print("Puntuacion final: ", sentimiento)

# También podemos ver el grado de sensibilidad en función de opiniones predominantes respecto del total
print("% sensibilidad: ", 100*max(abs(len(loc_posit)), abs(len(loc_neg)))/(len(loc_posit)+len(loc_neg)))

Puntuacion final:  2
% sensibilidad:  60.0
